In [1]:
import pandas as pd
import json
import gzip

In [2]:
file = "/home/marc/inters/output_test2/05_Annotation/Nirvana/snvs/Annotation.json.gz"

In [3]:

header = ''
positions = []
genes = []
is_header_line = True
is_position_line = False
is_gene_line = False
gene_section_line = '],"genes":['
end_line = ']}'

with gzip.open(file, 'rt') as f:
    position_count = 0
    gene_count = 0
    for line in f:
        trim_line = line.strip()
        if is_header_line:
            ## only keep the "header" field content from the line
            header = trim_line[10:-14]
            is_header_line = False
            is_position_line = True
            continue
        if trim_line == gene_section_line:
            is_gene_line = True
            is_position_line = False
            continue
        elif trim_line == end_line:
            break
        else:
            if is_position_line:
                ## remove the trailing ',' if there is
                positions.append(trim_line.rstrip(','))
                position_count += 1
            if is_gene_line:
                ## remove the trailing ',' if there is
                genes.append(trim_line.rstrip(','))
                gene_count += 1

print ('number of positions:', position_count)
print ('number of genes:', gene_count)


number of positions: 6557
number of genes: 313


In [16]:
mylist = []
for position in positions:
    position_dict = json.loads(position)
    mylist.append(position_dict)
df = pd.DataFrame(mylist)
variants_df = df['variants'].apply(pd.Series)
variants_df2 = variants_df[0].apply(pd.Series)
final_df = pd.concat([df, variants_df2], axis=1).drop(['variants', 'samples'], axis=1)



chromosome                                                            chr8
position                                                            196884
refAllele                                                                C
altAlleles                                                             [A]
quality                                                            13610.7
filters                                                             [PASS]
fisherStrandBias                                                     3.898
mappingQuality                                                          60
cytogeneticBand                                                     8p23.3
vid                                                           8-196884-C-A
chromosome                                                            chr8
begin                                                               196884
end                                                                 196884
refAllele                

In [5]:
mykeys = set()
for position in positions:
    position_dict = json.loads(position)
    if "variants" in position_dict:
        for variant_dict in position_dict["variants"]:
            for keys in variant_dict:
                mykeys.add(keys)
print(list(mykeys))

['revel', 'variantType', 'topmed', 'gnomad', 'clinvar', 'oneKg', 'gerpScore', 'vid', 'chromosome', 'regulatoryRegions', 'globalAllele', 'altAllele', 'refAllele', 'phylopScore', 'dbsnp', 'begin', 'end', 'dannScore', 'inLowComplexityRegion', 'transcripts', 'hgvsg']
